In [ ]:
# Imports
import pandas as pd
import os
from PIL import Image
import glob


In [ ]:
# Convert .xlsx to .csv
def xlsx_to_csv(excel_file: str, sheet_name: str, output_file: str) -> None:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    df.to_csv(output_file, index=False)
    print(f"Converted {excel_file} to {output_file}")


In [ ]:
excel_file = "./new_sorted.xlsx"
csv_file = "./sorted_decisions.csv"
sheet_name = "Sheet1"
xlsx_to_csv(excel_file, sheet_name, csv_file)


In [ ]:
# Fill empty fields
df = pd.read_csv(csv_file)
df['in_hand_item'].fillna('no_item', inplace=True)
df['target_mob'].fillna('no_mob', inplace=True)

df.to_csv(csv_file, index=False)


In [ ]:
# Compressing images
def compress_image(image_path, output_path, max_size=(800, 450), quality=85):
    try:
        with Image.open(image_path) as img:
            if img.mode == 'RGBA':
                img = img.convert('RGB')
                
            img.thumbnail(max_size)
            img.save(output_path, "JPEG", quality=quality)
            print(f"Image compressed and saved: {output_path}")
    except Exception as e:
        print(f"Error compressing {image_path}: {e}")

def compress_images_in_folder(input_folder, output_folder, max_size=(800, 450), quality=85):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        input_path = os.path.join(input_folder, filename)
        if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            output_path = os.path.join(output_folder, filename)
            compress_image(input_path, output_path, max_size, quality)

input_folder = "./raw_images"
output_folder = "./compressed_images"

compress_images_in_folder(input_folder, output_folder)


In [ ]:
# Crop only hearts and toolbar fragments
def process_screenshots(input_folder, hearts_output_folder, in_hand_output_folder):
    os.makedirs(hearts_output_folder, exist_ok=True)
    os.makedirs(in_hand_output_folder, exist_ok=True)

    heart_region_coords = (215, 370, 400,  420)
    in_hand_region_coords = (200, 410, 600,  450)

    screenshot_files = glob.glob(os.path.join(input_folder, '*.png'))

    for screenshot_path in screenshot_files:
        image = Image.open(screenshot_path).convert('RGB')

        heart_region = image.crop(heart_region_coords)
        heart_filename = os.path.join(hearts_output_folder, os.path.basename(screenshot_path))
        heart_region.save(heart_filename)

        in_hand_region = image.crop(in_hand_region_coords)
        in_hand_filename = os.path.join(in_hand_output_folder, os.path.basename(screenshot_path))
        in_hand_region.save(in_hand_filename)

        print(f"Processed {os.path.basename(screenshot_path)}")

input_folder = "./compressed_images"
hearts_output_folder = "./hearts_regions"
in_hand_output_folder = "./in_hand_regions"
process_screenshots(input_folder, hearts_output_folder, in_hand_output_folder)
